## Resident Assistant (RA) Duty Schedule Creator

Welcome to the RA Duty Schedule Creator!

In [1]:
import pandas as pd
import numpy as np

In [2]:
# these values would be inputted by the user

duty_start_date = '01-06-2024'
duty_end_date = '03-23-2024'

In [13]:
# let's load our dataset containing the availability of each RA for the quarter
# availability file also provided by user

avail = pd.read_csv('RA_availability.csv')
avail.head()

,code_name,Shifts,Unavailable
0,RA1,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",NaN
1,RA2,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",NaN
2,RA3,"Thursday 6 PM - 11 PM, Friday 6 PM - 11 PM, Sa...",01-28-2024
3,RA4,"Wednesday 6 PM - 11 PM, Friday 6 PM - 11 PM, S...",01-06-2024
4,RA5,"Tuesday 6 PM - 11 PM, Thursday 6 PM - 11 PM, S...",01-27-2024


In [14]:
# change column names
avail.columns = ['ra_name', 'availability', 'not_available']
avail.head()

,ra_name,availability,not_available
0,RA1,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",NaN
1,RA2,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",NaN
2,RA3,"Thursday 6 PM - 11 PM, Friday 6 PM - 11 PM, Sa...",01-28-2024
3,RA4,"Wednesday 6 PM - 11 PM, Friday 6 PM - 11 PM, S...",01-06-2024
4,RA5,"Tuesday 6 PM - 11 PM, Thursday 6 PM - 11 PM, S...",01-27-2024


Column Descriptions:
- ra_name: Name of the RA. For data privacy reasons we have encoded each name as 'RA#number'
- availability: Indicates which shifts each RA can work. There are are 11 possible shifts. Monday to Friday 6 PM - 11 PM shift. On Saturday and Sunday, 9 AM - 2 PM, 2 PM - 7 PM, and 7 PM - 11 PM shifts.
- not_available: Allows each RA to list specific dates they are unavailable to work.

In [15]:
# format the not_available column as a list of dates
not_available_dates = avail.apply(lambda x: [] if pd.isna(x['not_available']) \
                                  else [i.strip() for i in x["not_available"].strip().split(", ")], axis = 1)
avail = avail.assign(not_available=not_available_dates)
avail.head()

,ra_name,availability,not_available
0,RA1,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",[]
1,RA2,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",[]
2,RA3,"Thursday 6 PM - 11 PM, Friday 6 PM - 11 PM, Sa...",[01-28-2024]
3,RA4,"Wednesday 6 PM - 11 PM, Friday 6 PM - 11 PM, S...",[01-06-2024]
4,RA5,"Tuesday 6 PM - 11 PM, Thursday 6 PM - 11 PM, S...",[01-27-2024]


In [16]:
# expand the availability column
shifts = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday 9', 'Saturday 2', 'Saturday 7', 'Sunday 9', 'Sunday 2', 'Sunday 7']
for shift in shifts:
    shift_avail = avail['availability'].apply(lambda x : True if shift in x else False)
    avail[shift] = shift_avail

In [18]:
# readjusting column names
avail.columns = ['ra_name', 'availability', 'not_available', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', \
                 'Saturday1', 'Saturday2', 'Saturday3', 'Sunday1', 'Sunday2', 'Sunday3']
avail.head()

,ra_name,availability,not_available,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday1,Saturday2,Saturday3,Sunday1,Sunday2,Sunday3
0,RA1,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",[],True,True,True,False,False,True,True,False,True,True,True
1,RA2,"Monday 6 PM - 11 PM, Tuesday 6 PM - 11 PM, Wed...",[],True,True,True,True,True,True,True,True,True,True,True
2,RA3,"Thursday 6 PM - 11 PM, Friday 6 PM - 11 PM, Sa...",[01-28-2024],False,False,False,True,True,True,True,True,True,True,True
3,RA4,"Wednesday 6 PM - 11 PM, Friday 6 PM - 11 PM, S...",[01-06-2024],False,False,True,False,True,True,True,True,True,True,True
4,RA5,"Tuesday 6 PM - 11 PM, Thursday 6 PM - 11 PM, S...",[01-27-2024],False,True,False,True,False,True,False,True,True,False,True


The day columns indicate whether a RA is available to work the shift on that day. 'Saturday1' denotes the first shift of Saturday. The same applies for the other weekend shifts.

In [8]:
# identify which days are holidays
# webscrape from the registar's website https://blink.ucsd.edu/instructors/resources/academic/calendars/2023.html

import requests
import bs4
import warnings

warnings.filterwarnings('ignore')

reg_website = requests.get('https://blink.ucsd.edu/instructors/resources/academic/calendars/2023.html', verify=False)
reg_website_text = reg_website.text
reg_website_soup = bs4.BeautifulSoup(reg_website_text)
reg_soup_text = reg_website_soup.text

print(type(reg_soup_text))
print(len(reg_soup_text))
print(reg_soup_text[1000: 3000])

<class 'str'>
8466
t, Health & Safety
Police Department
Operations Management and Capital Programs




Blink Resources     

About Blink
Campus Directory
Departments in Blink
List of Tools







Budget & Finance

BI & Financial Reporting
Buy & Pay
Finance
Logistics
Finance Administration Resources
Travel & Entertainment


Facilities & Services 

Campus Planning
Facilities Management
Real Estate
Triton Print & Digital Media
Transportation Services


Faculty & Instruction

Faculty
Student/ Class Info
Instruction Tools


Human Resources 

About Us
Benefits
Ecotime
Employee Center
UCPath
UC Learning Center


IT Services

Get Help
Technology Topics
Browse Service Portal
Connect from Home
Careers
About Us


Research & Innovation

About Us
Organized Research Units (ORUs)
Research Affairs
Sponsored Research Administration


Safety 

Environment, Health & Safety
Police Department
Operations Management and Capital Programs


Blink Resources

About Blink
Campus Directory
Departments in Blink
Lis

In [9]:
# search for holidays
# the date list mentioned below would change with the quarter

import re

jan_dates = re.findall('Holiday\n.+January.+\n', reg_soup_text)
feb_dates = re.findall('Holiday\n.+February.+\n', reg_soup_text)

print('January Holiday Dates:', jan_dates)
print('February Holiday Dates:', feb_dates)

January Holiday Dates: ['Holiday\nMonday - Tuesday, January 1-2, 2024\n', 'Holiday\nMonday, January 15\n']
February Holiday Dates: ['Holiday\nMonday, February 19\n']


In [10]:
# converting list of dates

holidays = ['01-15-2024', '02-19-2024']

In [11]:
# generate date range

dates = pd.date_range(start=duty_start_date, end =duty_end_date, freq ='1D')
dates = list(dates)
dates = [pd.to_datetime(i).strftime('%m-%d-%Y') for i in dates]

# remove any holidays from the date range
for holiday in holidays:
    dates.remove(holiday)

print('First 10 entries:', dates[0:10])
print('End date:', dates[-1])

First 10 entries: ['01-06-2024', '01-07-2024', '01-08-2024', '01-09-2024', '01-10-2024', '01-11-2024', '01-12-2024', '01-13-2024', '01-14-2024', '01-16-2024']
End date: 03-23-2024


In [22]:
# creating date and day columns for our schedule dataset
date_col = []
day_col = []

for date in dates:
    date_col.append(date)

    if pd.to_datetime(date).day_name() in ['Saturday', 'Sunday']:
        date_col.append(date)
        date_col.append(date)

        day_col.append(pd.to_datetime(date).day_name() + '1')
        day_col.append(pd.to_datetime(date).day_name() + '2')
        day_col.append(pd.to_datetime(date).day_name() + '3')

    else:
        day_col.append(pd.to_datetime(date).day_name())

print('First 10 entries in date_col:', date_col[0:11])
print('First 10 entries in day_col:', day_col[0:11])
print(len(date_col))
print(len(day_col))

First 10 entries in date_col: ['01-06-2024', '01-06-2024', '01-06-2024', '01-07-2024', '01-07-2024', '01-07-2024', '01-08-2024', '01-09-2024', '01-10-2024', '01-11-2024', '01-12-2024']
First 10 entries in day_col: ['Saturday1', 'Saturday2', 'Saturday3', 'Sunday1', 'Sunday2', 'Sunday3', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
122
122


In [23]:
# create the schedule dataframe

schedule = pd.DataFrame({'Date' : date_col, 'Day' : day_col})
schedule.head()

,Date,Day
0,01-06-2024,Saturday1
1,01-06-2024,Saturday2
2,01-06-2024,Saturday3
3,01-07-2024,Sunday1
4,01-07-2024,Sunday2


In [45]:
# count shifts per RA
import random

num_RAs = avail.shape[0]
# multiply by two to account for a pair of RAs working each shift
num_shifts = schedule.shape[0] * 2
shift_dist = {}
base_shifts = num_shifts // num_RAs
num_extras = num_shifts - base_shifts * num_RAs

# randomly choose RAs to work any remaining shifts after equal distribution
extra_RA = random.sample(range(1,43), num_extras)

RA_names = [f'RA{i}' for i in range(1, 43)]

for ra in RA_names:
    shift_dist[ra] = 5
    if int(ra[2:]) in extra_RA:
        shift_dist[ra] = 6

print('Shift distribution count for each RA:', shift_dist)

Shift distribution count for each RA: {'RA1': 6, 'RA2': 6, 'RA3': 6, 'RA4': 6, 'RA5': 6, 'RA6': 6, 'RA7': 5, 'RA8': 5, 'RA9': 6, 'RA10': 6, 'RA11': 5, 'RA12': 6, 'RA13': 6, 'RA14': 6, 'RA15': 6, 'RA16': 6, 'RA17': 6, 'RA18': 6, 'RA19': 6, 'RA20': 6, 'RA21': 6, 'RA22': 5, 'RA23': 6, 'RA24': 6, 'RA25': 5, 'RA26': 5, 'RA27': 6, 'RA28': 6, 'RA29': 5, 'RA30': 6, 'RA31': 6, 'RA32': 6, 'RA33': 6, 'RA34': 6, 'RA35': 6, 'RA36': 6, 'RA37': 6, 'RA38': 6, 'RA39': 6, 'RA40': 6, 'RA41': 5, 'RA42': 6}


In [49]:
# verfication, expected number of shifts is 244

assert sum(shift_dist.values()) == num_shifts
print("Total number of shifts:", sum(shift_dist.values()))

Total number of shifts: 244


The days on which duty shifts occur, the number of shifts each day, and the number of shifts per RA have been calculated now. Next, to assign each RA to a shift based on their availability, we will randomly assign shifts to each RA then swap positions of RAs to correct their placement based on availability.

In [77]:
# randomly assign shifts 
# assign RA pair to each shift

first_RA = [f'RA{i}' for i in extra_RA] * 3
second_RA = first_RA.copy()

# find RAs with 5 shifts
five_RA = [f'RA{i}' for i in range(1, 43) if i not in extra_RA]
first_half = five_RA[0 : int(len(five_RA) / 2)]
second_half = five_RA[int(len(five_RA) / 2) : ]

first_RA = first_RA + (first_half * 3) + (second_half * 2)
second_RA = second_RA + (first_half * 2) + (second_half * 3)

first_RA = random.sample(first_RA, len(first_RA))
second_RA = random.sample(second_RA, len(second_RA))

In [78]:
# verfication steps

assert len(first_RA) == schedule.shape[0]
assert len(second_RA) == schedule.shape[0]

# shift distribution matches number of randomly assigned shifts
for ra in RA_names:
    assert first_RA.count(ra) + second_RA.count(ra) == shift_dist[ra]

In [79]:
# create random schedule

schedule = schedule.assign(First_RA = first_RA)
schedule = schedule.assign(Second_RA = second_RA)

schedule

,Date,Day,First_RA,Second_RA
0,01-06-2024,Saturday1,RA11,RA31
1,01-06-2024,Saturday2,RA37,RA36
2,01-06-2024,Saturday3,RA1,RA16
3,01-07-2024,Sunday1,RA8,RA39
4,01-07-2024,Sunday2,RA1,RA34
...,...,...,...,...
117,03-21-2024,Thursday,RA16,RA21
118,03-22-2024,Friday,RA7,RA19
119,03-23-2024,Saturday1,RA22,RA20
120,03-23-2024,Saturday2,RA23,RA23


The above duty schedule is randomly generated. In the dataframe, one row corresponds to one shift while each column refers to specific details of that shift, i.e., pair of RAs on duty, the day of the shift, and shift timings. 

To reiterate some details, 'RA#number' refers to the full name (must be unique) of a RA. Since the current demo has a group of 42 RAs, the names (or code names) range from RA1 to RA42. The First_RA and Second_RA column specify which pair of RAs will be serving each shift. The day columns specifies the time of the shift and the day the shift is occurring. These have been merged for convenience. 'Saturday#number' refers to the time of the shift on Saturday, for instance, the first shift on Saturday or Saturday1 is from 9 AM to 2 PM. Monday to Friday contain no numbers since the assumption here is that there is only one shift each day from Monday to Friday. These details can be adjusted by the user, i.e. the user can specify the number of shifts each day.

In [85]:
# iterate through schedule to correct positions of RA based on availability
# if RA availability does not match with randomly assigned shift then search for RAs above whose availability matches and swap shifts
# correcting first column

for i in range(0, len(schedule)):
    ra = schedule.iloc[i]['First_RA']
    ra_idx = int(ra[2:]) - 1
    day = schedule.iloc[i]['Day']
    date = schedule.iloc[i]['Date']

    not_available = avail.iloc[ra_idx]['not_available']
    
    if ((avail.iloc[ra_idx][day] == False) or (date in not_available)):
        swap = True
        above_idx = i - 1
        count = 0

        while (swap):

            if (count > len(schedule)):
                print("Cannot place RA", ra, date)
                break

            if (above_idx == -1):
                above_idx = len(schedule) - 1

            ra_above = schedule.iloc[above_idx]['First_RA']
            ra_above_idx = int(ra_above[2:]) - 1
            day_above = schedule.iloc[above_idx]['Day']
            ra_above_not_available = avail.iloc[ra_above_idx]['not_available']
            date_above = schedule.iloc[above_idx]['Date']

            if ((avail.iloc[ra_above_idx][day]) and (avail.iloc[ra_idx][day_above]) and (date_above not in not_available) and (date not in ra_above_not_available)):

                temp = schedule['First_RA'][i]
                schedule['First_RA'][i] = schedule['First_RA'][above_idx]
                schedule['First_RA'][above_idx] = temp

                swap = False
            else:
                above_idx = above_idx - 1
                count = count + 1

In [89]:
# correcting the second column

for i in range(0, len(schedule)):
    ra1 = schedule.iloc[i]['First_RA']
    ra1_idx = int(ra1[2:]) - 1

    ra2 = schedule.iloc[i]['Second_RA']
    ra2_idx = int(ra2[2:]) - 1

    day = schedule.iloc[i]['Day']
    date = schedule.iloc[i]['Date']

    ra2_not_available = avail.iloc[ra2_idx]['not_available']

    if ((ra1 == ra2) or (avail.iloc[ra2_idx][day] == False) or (date in ra2_not_available)):
        #linear probe backwards 
        swap = True
        above_idx = i - 1
        count = 0

        while (swap):

            if (count > len(schedule)):
                print("Cannot place", ra2, date)
                break

            if (above_idx == -1):
                above_idx = len(schedule) - 1

            ra2_above = schedule.iloc[above_idx]['Second_RA']
            ra2_above_idx = int(ra2_above[2:]) - 1

            ra1_above = schedule.iloc[above_idx]['First_RA']
            ra1_above_idx = int(ra1_above[2:]) - 1

            day_above = schedule.iloc[above_idx]['Day']
            date_above = schedule.iloc[above_idx]['Date']

            ra2_above_not_available = avail.iloc[ra2_above_idx]['not_available']

            if ((date_above in ra2_not_available) or (date in ra2_above_not_available)):
                count = count + 1
                above_idx = above_idx - 1
                continue

            if ((avail.iloc[ra2_above_idx][day]) and (avail.iloc[ra2_idx][day_above]) and (ra2_above != ra1) and (ra1_above != ra2)):
                temp = schedule['Second_RA'][i]
                schedule['Second_RA'][i] = schedule['Second_RA'][above_idx]
                schedule['Second_RA'][above_idx] = temp

                swap = False
            else:
                above_idx = above_idx - 1
                count = count + 1

In [90]:
# here is the final schedule
schedule

,Date,Day,First_RA,Second_RA
0,01-06-2024,Saturday1,RA11,RA36
1,01-06-2024,Saturday2,RA17,RA31
2,01-06-2024,Saturday3,RA8,RA40
3,01-07-2024,Sunday1,RA1,RA30
4,01-07-2024,Sunday2,RA11,RA34
...,...,...,...,...
117,03-21-2024,Thursday,RA16,RA39
118,03-22-2024,Friday,RA27,RA23
119,03-23-2024,Saturday1,RA9,RA14
120,03-23-2024,Saturday2,RA23,RA35


In [95]:
# verify the shifts assigned

for i in range(122):
    day = schedule.iloc[i]['Day']

    ra1 = schedule.iloc[i]['First_RA']
    ra1_idx = int(ra1[2:]) - 1

    ra2 = schedule.iloc[i]['Second_RA']
    ra2_idx = int(ra2[2:]) - 1

    assert avail.iloc[ra1_idx][day]
    assert avail.iloc[ra2_idx][day]

In [96]:
# convert to csv file
schedule.to_csv('RA-duty-schedule', index=False)

And we're done!